# Thumbnail creation
input: dimensionality reduced audio features
output: thumbnail images

## Import libraries and (filepaths to) audio (features)

In [ ]:
import pandas as pd
import librosa
import importlib
from IPython.display import display, Audio
from datetime import datetime


import thumbnail_canvas_functions as tcf
importlib.reload(tcf)

pd.set_option('display.float_format', lambda x: f'{x:.3f}')

In [ ]:
mp3_folder = '../user_evaluation_app/static/mp3_previews'
audio_features_loc = 'dimensionality_reduced.csv'

today = datetime.now().strftime('%Y%m%d')

# output folder base name
output_folder = f'{today}_thumbnails_tsne'

In [ ]:
audio_features = pd.read_csv(audio_features_loc, index_col='track_id')
audio_features

## Create the radar chart thumbnail

In [ ]:
instrument_features = audio_features.filter(regex='^i', axis=1)
instrument_labels_list = ['i voice','i choir','i flute', 'i strings','i harp', 'i piano','i guitar','i drums', 'i synth']
# Remove the 'i ' prefix from each element
instrument_labels_list_without_prefix = [label[2:] if label.startswith('i ') else label for label in instrument_labels_list]

In [ ]:
def display_audio(id, show_metadata = True, show_song = True, metadata = audio_features, mp3_folder = mp3_folder):
    if show_metadata:
        song_metadata = metadata.loc[id]
        artist_name = song_metadata.loc['spotify artist_name']
        track_name = song_metadata.loc['spotify track_name']
        print(f"Artist: {artist_name}, Track: {track_name} \n ID: {id}")
        #print(metadata.loc[id])
    if show_song:
        y, sr = librosa.load(f'{mp3_folder}/{id}.mp3')
        display(Audio(data=y, rate=sr))
    return None

# Create genre colors to fill the radar chart

In [ ]:
genre_features = audio_features.filter(regex='^g', axis=1)


In [ ]:
import colorsys

# Function to convert HSV to RGB and then to hexadecimal
def hsv_to_rgb(row):
    h, s, v = row['g TSNE1']*0.9, row['g TSNE2'], 1- (row['g TSNE2']* 0.6)
    r, g, b = colorsys.hsv_to_rgb(h, s, v)
    return r, g, b

import colorsys

genre_features['g rgb color'] = pd.DataFrame(genre_features.apply(lambda row: hsv_to_rgb(row), axis = 1))

genre_features


In [ ]:
audio_features['g rgb color'] = genre_features['g rgb color']
audio_features

# Create SoundShapes

In [ ]:
audio_features.apply(lambda x: tcf.create_thumbnail(x, display=False, size=600, debug=False, dest_folder=f'{output_folder}_{len(audio_features.index)}'), axis=1)
# 
print(f'{len(audio_features.index)} thumbnails created and stored in output folder starting with {output_folder}_{len(audio_features.index)}')